In [1]:
from rdflib import Graph

In [2]:
g = Graph()
g.parse("../rdf/lw-eii.ttl", format="ttl")

BadSyntax: at line 40 of <>:
Bad syntax (expected '.' or '}' or ']' at end of statement) at ^ in:
"...b'i:netCDF ;\r\n  rdfs:label "netCDF CF conventions" ;\r\n.\r\neii:L'^b'&W-Digital-Asset-Registry rdf:type eii:Technology ;\r\n  rdfs:'..."

In [ ]:
len(g) # prints 2

import pprint
for stmt in g:
    pprint.pprint(stmt)


In [ ]:
qres = g.query(
    """PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       SELECT DISTINCT ?x ?p ?y ?xtype ?ytype
       WHERE {
          ?a ?p ?b .
          ?a rdfs:label ?x .
          ?a a ?xtype .
          ?b rdfs:label ?y .
          ?b a ?ytype .
       }""")
print(qres)


In [ ]:
thingIdx = {}     #dict of things
thingTypeIdx = {} #dict of array of things
thingTypeReverseIdx = {} #dict of array of things

relationList = []  #dict of relationships between things

for (x,p,y,xtype,ytype) in qres:
    print(x)
    print(g.qname(xtype))
    subject = str(x)
    subject_type = str(g.qname(xtype))
    
    print(g.qname(p))
    prop = str(g.qname(p))
    
    print(y)
    print(g.qname(ytype))
    obj = str(y)
    obj_type = str(g.qname(ytype))
    print("\n")
    
    #add things to thingIdx to ensure unique things
    if(subject not in thingIdx):
        thingIdx[subject] = 1
    else:
        #might as well increment count of occurences
        thingIdx[subject] = thingIdx[subject] + 1
    
    if(obj not in thingIdx):
        thingIdx[obj] = 1
    else:
        #might as well increment count of occurences
        thingIdx[obj] = thingIdx[obj] + 1
    
    #add things to thingTypeIdx
    if(subject_type not in thingTypeIdx):
        thingTypeIdx[subject_type] = {}
    thingTypeIdx[subject_type][subject] = 1
    
    if(subject not in thingTypeReverseIdx):
        thingTypeReverseIdx[subject] = {}
    thingTypeReverseIdx[subject][subject_type] = 1
    
    if(obj_type not in thingTypeIdx):
        thingTypeIdx[obj_type] = {}
    thingTypeIdx[obj_type][obj] = 1
    
    if(obj not in thingTypeReverseIdx):
        thingTypeReverseIdx[obj] = {}
    thingTypeReverseIdx[obj][obj_type] = 1
    
    #add relations
    rel = {
        "source" : subject,
        "target" : obj,
        "value"  : '1',
        "type"   :  prop
    }
    relationList.append(rel)
    

In [ ]:
pprint.pprint(thingIdx)

In [ ]:
pprint.pprint(thingTypeIdx)

In [ ]:
pprint.pprint(thingTypeReverseIdx)

In [ ]:
pprint.pprint(relationList)

In [ ]:
#create index out of types
#get types
typeList = thingTypeIdx.keys()
typeIdx = {}
nodes = []
counter = 1
for t in typeList:
    typeIdx[t] = counter
    counter = counter + 1
    
for obj in thingTypeReverseIdx.keys():
    typeIds = []
    arrTypeLabels = []
    for x in thingTypeReverseIdx[obj].keys():
        typeIds.append(typeIdx[x])
        arrTypeLabels.append(x)
    #obj = { "id" : id, "group": typeIds }
    #obj = { "id" : id, "group": arrTypeLabels }
    group = 1
    if('prov:Person' in arrTypeLabels):
        group = 10
    if('prov:Organization' in arrTypeLabels):
        group = 5
    if('eii:Technology' in arrTypeLabels):
        group = 7
        
    d = { "id" : obj, "group": group , "type" : arrTypeLabels }
    
    nodes.append(d)

pprint.pprint(nodes)
pprint.pprint(typeList)
pprint.pprint(typeIdx)
               

In [ ]:
d3_obj = { "nodes" : nodes, "links" : relationList}
pprint.pprint(d3_obj)

In [ ]:
import json
with open('data.json', 'w') as outfile:
    json.dump(d3_obj, outfile, sort_keys = True, indent = 4)